In [2]:
from bce import * 
import pandas as pd
import numpy as np
import gc
from sklearn.preprocessing import StandardScaler

In [22]:
df_train = data_clean('./bihar_features_gee.csv')
df_test = pd.read_csv('./up_features_5km_04102018.csv')
x_train, y_train = df_train['df'][0], df_train['label'][0]

Begin data cleaning.
Read in data from ./bihar_features_gee.csv sucessfully.
All dataframes and labels generated.
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Column Percentage Electrified does not exist
Column Number of Electrified Households does not exist
Column Unnamed: 0 does not exist
Column Census 2011 ID does not exist
Column Number_of_Households does not exist
Deleting useless and predictive informative columns
Data cleaning successful.
NaN left: []
NaN left: []
NaN left: []


In [23]:
col_dict_1 = {}
for i in x_train.columns:
    col_dict_1[i] = i.upper()
x_train = x_train.rename(index=str, columns=col_dict_1)

In [24]:
col_dict_2 = {}
for i in df_test.columns:
    col_dict_2[i] = i.upper()
df_test = df_test.rename(index=str, columns=col_dict_2)

In [25]:
scaler = StandardScaler()
features = [i for i in df_test.columns if i != 'label' and i in x_train.columns]

In [26]:
len(features)

274

In [27]:
x_train = scaler.fit_transform(x_train[features])
x_test = scaler.fit_transform(df_test[features].fillna(-1))

In [28]:
gdb = GradientBoostingClassifier(n_estimators=7, 
                                 max_depth=6,
                                 min_samples_split=1780, 
                                 min_samples_leaf=1,
                                 random_state=20180320,
                                 subsample=0.8, 
                                 learning_rate=0.11)
        
weighting = lambda x:1 if x else 20
gdb.fit(x_train, y_train, sample_weight=[weighting(i) for i in y_train])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.11, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=1780,
              min_weight_fraction_leaf=0.0, n_estimators=7, presort='auto',
              random_state=20180320, subsample=0.8, verbose=0,
              warm_start=False)

In [29]:
predictions = gdb.predict(x_test)
df_test = df_test.assign(prediction=predictions)

In [36]:
df_test[['SYSTEM:INDEX', 'SYSTEM_IND', 'prediction']].to_csv('./prediction.csv')

In [35]:
for i in df_test.columns:
    if any(j in i for j in ['ID', 'INDEX', "SYSTEM"]):
        print(i)

SYSTEM:INDEX
SYSTEM_IND
